In [1]:
from scipy.spatial import KDTree
from geopy.distance import geodesic
import pandas as pd

##### DATA SOURCE: https://data.ibb.gov.tr/en/dataset/bisiklet-ve-mikromobilite-park-alanlari/resource/f6f9a6af-84d6-4718-b509-4a9ccfba038f

In [4]:
gh_df = (
    pd.read_csv("datasets/15_gh_info/geohash_traffic_density_pt_15clusters.csv", usecols=['GEOHASH', 'LATITUDE', 'LONGITUDE'])
)

gh_df.head()

,GEOHASH,LATITUDE,LONGITUDE
0,sxk3xw,41.080627,28.811646
1,sxk9nm,40.987244,29.108276
2,sxk9q0,41.003723,29.097290
3,sxk3hx,40.998230,28.679810
4,sx7cmx,41.042175,28.020630


In [8]:
bike_mm_parking = (
    pd.read_excel("datasets/17_bike_micromobility_oarking_areas/bisiklet-ve-mikromobilite-park-alanlar.xlsx",
                  usecols=['Park Alanı Adı', 'X Koordinatı', 'Y Koordinatı'])
    .rename(columns={'X Koordinatı' : 'longitude', 'Y Koordinatı' : 'latitude'})
)

bike_mm_parking.head()

,Park Alanı Adı,longitude,latitude
0,Siteler Metro İstasyonu,28.796330,41.088241
1,Başak Konutları Metro İstasyonu,28.791287,41.097669
2,Başakşehir Metrokent Metro İstasyonu,28.801564,41.107565
3,Ziya Gökalp Metro İstasyonu,28.786672,41.074669
4,Olimpiyat Metro İstasyonu,28.767234,41.079466


In [10]:
site_dict = (
    bike_mm_parking
    .set_index('Park Alanı Adı')
    .T
    .to_dict()
)

In [11]:
# Unique list of coordinates
coordinates = gh_df[['LATITUDE', 'LONGITUDE']].drop_duplicates().values
coordinates

array([[41.080627, 28.811646],
       [40.987244, 29.108276],
       [41.003723, 29.09729 ],
       ...,
       [40.976257, 29.229126],
       [41.009216, 27.98767 ],
       [41.020203, 27.998657]])

In [12]:
def find_nearest_coordinates(
        target_lat: float,
        target_lon: float,
        coordinates: list[list[float, float]]
        ) -> list[tuple[tuple[float, float], float]]:
    """
    Finds the nearest coordinates to a target latitude and longitude.

    Args:
        target_lat (float): The latitude of the target location.
        target_lon (float): The longitude of the target location.
        coordinates (List[Tuple[float, float]]): A list of tuples representing the coordinates to search from.

    Returns:
        List[Tuple[Tuple[float, float], float]]: A list of tuples containing the nearest coordinates and their distances in kilometers.
    """
    target_coords = (target_lat, target_lon)
    
    # Build a KDTree for efficient nearest neighbor search
    tree = KDTree(coordinates)

    # Calculate the search radius in degrees
    radius_deg = 1 / 111.0  # 1 km radius in degrees

    # Query the KDTree for indices of points within the search radius
    indices = tree.query_ball_point([(target_lat, target_lon)], radius_deg)

    # Calculate the distance between target coordinates and nearest coordinates
    nearest_coordinates = [(coordinates[i], geodesic(target_coords, coordinates[i]).km) for i in indices[0]]

    # Sort the nearest coordinates by distance
    nearest_coordinates.sort(key=lambda x: x[1])  # Sort by distance

    return nearest_coordinates

In [14]:
bike_mm_parking_list = []
new_list = []

for site in site_dict:

    target_lat = site_dict[site]['latitude']
    target_lon = site_dict[site]['longitude']

    nearest_coords = find_nearest_coordinates(target_lat, target_lon, coordinates)

    for i, (coords, distance) in enumerate(nearest_coords):

        gh = gh_df.loc[gh_df['LATITUDE'].eq(coords[0]) & gh_df['LONGITUDE'].eq(coords[1]), 'GEOHASH'].values[0]

        # Creating two dataframe which will later be merged rows wise for visualization purpose.
        bike_mm_parking_list.append({
            "type" : "bike_mm_parking",
            "name" : site,
            "latitude" : target_lat,
            "longitude" : target_lon,
            "nearest" : site
        })

        new_list.append({
            "type" : "geohash",
            "name" : gh,
            "latitude" : coords[0],
            "longitude" : coords[1],
            "nearest" : site
        })

In [16]:
df = (
    pd.concat([pd.DataFrame(new_list), pd.DataFrame(bike_mm_parking_list)], ignore_index=True)
    .drop_duplicates(ignore_index=True)
)

df

,type,name,latitude,longitude,nearest
0,geohash,sxk3xr,41.086120,28.800660,Siteler Metro İstasyonu
1,geohash,sxk3z2,41.091614,28.800660,Siteler Metro İstasyonu
2,geohash,sxk3xp,41.086120,28.789673,Siteler Metro İstasyonu
3,geohash,sxk3z0,41.091614,28.789673,Siteler Metro İstasyonu
4,geohash,sxk3xq,41.080627,28.800660,Siteler Metro İstasyonu
...,...,...,...,...,...
1081,bike_mm_parking,Haliç Metro İstasyonu,41.020250,28.964381,Haliç Metro İstasyonu
1082,bike_mm_parking,Taş Mektep,40.968969,29.071657,Taş Mektep
1083,bike_mm_parking,Söğütlüçeşme İSPARK - 1,40.991669,29.038443,Söğütlüçeşme İSPARK - 1
1084,bike_mm_parking,Şimşek Sokak,40.969521,29.063754,Şimşek Sokak


In [17]:
# Some GEOHASH has up to 14 biking/micromobility parking within its 1km radius
(
    df.query("type=='geohash'")
    .groupby('name')['nearest'].nunique()
    .nlargest(10)
)

name
sxk9je    14
sxk9ee    13
sxk977    12
sxk9es    12
sxk9jb    12
sxk976    11
sxk9eg    11
sxk9eu    11
sxk9jc    11
sxk9j9    10
Name: nearest, dtype: int64

In [18]:
df.to_csv("datasets/13_gh_proximities/gh_nearby_bike_mm_parking.csv", index=False)